In [1]:
!unzip hw2.zip

Streaming output truncated to the last 5000 lines.
  inflating: hw2/unlabeled/1440.jpg  
  inflating: hw2/unlabeled/0074.jpg  
  inflating: hw2/unlabeled/6348.jpg  
  inflating: hw2/unlabeled/0409.jpg  
  inflating: hw2/unlabeled/4875.jpg  
  inflating: hw2/unlabeled/2044.jpg  
  inflating: hw2/unlabeled/1670.jpg  
  inflating: hw2/unlabeled/2259.jpg  
  inflating: hw2/unlabeled/5619.jpg  
  inflating: hw2/unlabeled/6354.jpg  
  inflating: hw2/unlabeled/6938.jpg  
  inflating: hw2/unlabeled/7069.jpg  
  inflating: hw2/unlabeled/1506.jpg  
  inflating: hw2/unlabeled/6936.jpg  
  inflating: hw2/unlabeled/6041.jpg  
  inflating: hw2/unlabeled/4839.jpg  
  inflating: hw2/unlabeled/5989.jpg  
  inflating: hw2/unlabeled/6732.jpg  
  inflating: hw2/unlabeled/1598.jpg  
  inflating: hw2/unlabeled/2363.jpg  
  inflating: hw2/unlabeled/3908.jpg  
  inflating: hw2/unlabeled/1543.jpg  
  inflating: hw2/unlabeled/6896.jpg  
  inflating: hw2/unlabeled/1844.jpg  
  inflating: hw2/unlabeled/2913.jpg  

## 1. Image Preprocessing and Augmentation

In [2]:
from IPython.display import display
from PIL import Image
from pathlib import Path
import numpy as np
import pandas as pd
import shutil, time, os, requests, random, copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, models
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.manifold import TSNE

In [6]:
class Config:

    def __init__(self):

        self.batch_size = 128
        self.nb_epochs_per_saving = 1
        self.pin_mem = True
        self.num_cpu_workers = 2
        self.nb_epochs = 200
        self.cuda = True
        # Optimizer
        self.lr = 1e-4
        self.weight_decay = 5e-5
        # Hyperparameters for our y-Aware InfoNCE Loss
        self.sigma = 5 # depends on the meta-data at hand
        self.temperature = 0.1
        self.tf = "all_tf"
        self.model = "ResNet50"


        self.input_size = (3, 96, 96)

        self.checkpoint_dir = "save1/"

config = Config()      

In [7]:
TRAIN =Path("hw2/unlabeled")
TEST =Path("hw2/test")
#BATCH_SIZE = 256;

train_transform_set = [ 
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.5,contrast=0.5,saturation=0.5,hue=0.1)], p=0.8),
    #transforms.Pad(padding=(0, 0, 8, 16), fill=128),
    #transforms.GaussianBlur(kernel_size=9),
    
]
train_transform = transforms.Compose([
    transforms.RandomOrder(train_transform_set),
    #transforms.Grayscale(1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
test_transform = transforms.Compose([ 
    #transforms.Grayscale(1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [8]:
import natsort
from torch.utils.data import DataLoader, Dataset, RandomSampler
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform, mode):
        self.main_dir = main_dir
        self.transform = transform
        if mode=="train":
          all_imgs = os.listdir(main_dir)
        elif mode=="test":
          all_imgs=[]
          for path, subdirs, files in os.walk(main_dir):
              for name in files:
                  all_imgs.append(os.path.join(path, name))
        self.total_imgs = natsort.natsorted(all_imgs)
        self.mode = mode
    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        if self.mode=="train":
          img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        elif self.mode=="test":
          img_loc=self.total_imgs[idx]
        image = Image.open(img_loc)#.convert("RGB")
        
        if self.mode=="test":
          """labels = os.path.basename(os.path.dirname(img_loc)) 
          x = self.transform(image)
          x = torch.from_numpy(np.asarray(x))
          x = x.reshape(1, x.shape[0], x.shape[1], x.shape[2])
          labels = torch.from_numpy(np.asarray(int(labels)))
          return (x, labels)"""
          x1 = self.transform(image)
          x2 = self.transform(image)
          x = np.stack((x1, x2), axis=0)
          x = torch.from_numpy(np.asarray(x))
          return x
        else:
          x1 = self.transform(image)
          x2 = self.transform(image)
          x = np.stack((x1, x2), axis=0)
          x = torch.from_numpy(np.asarray(x))
          return x

train_set = CustomDataSet("hw2/unlabeled", transform=train_transform, mode = "train")
test_set = CustomDataSet("hw2/test", transform=test_transform, mode = "test")

train_loader = DataLoader(
    train_set,
    batch_size=config.batch_size,
    sampler=RandomSampler(train_set),
    #collate_fn=train_set.collate_fn,
    pin_memory=config.pin_mem,
    num_workers=config.num_cpu_workers
                              
)
val_loader = DataLoader(
    test_set,
    batch_size=config.batch_size,
    sampler=RandomSampler(test_set),
    #collate_fn=test_set.collate_fn,
    pin_memory=config.pin_mem,
    num_workers=config.num_cpu_workers
)

## 2. Model Definition


In [9]:
import torch
import torch.nn as nn

class ResNet(nn.Module):

    def __init__(self, num_classes, in_channels=1, depth=5, mode="simCLR"):
        """
        num_classes: int
            the number of features in the output segmentation map (in 'seg' mode) or
            number of classes (in 'classif' mode)
        in_channels: int, default 1
            number of channels in the input tensor.
        depth: int, default 5
            number of layers in the U-Net.
        mode: 'str', default 'seg'
            Whether the network is turned in 'segmentation' mode ("seg"), 'classification' ("classif"),
            'encoder' (only encoder pathway) or 'simCLR' (encoder pathway and non-linear head projection)
        """
        super(ResNet, self).__init__()
        self.mode = mode
        
        self.start_filts = 16 # Initial nb of kernels
        self.down_mode = "maxpool"
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.depth = depth
        self.classifier = None # Useful in classif mode
        self.name = "Resnet_D%i_%s" % (self.depth, self.mode)

        self.encoder = torchvision.models.resnet50(pretrained=False,num_classes=512)


        
        if self.mode == "classif":
            #self.classifier = Classifier(self.num_classes, features=self.start_filts * 2**(self.depth-1))
            self.fc1 = nn.Linear(512, 512, bias=True)
            self.fc2 = nn.Linear(512, self.num_classes, bias=True)
            self.relu = nn.LeakyReLU(inplace=True)

        elif self.mode == 'simCLR':
            self.hidden_representation = nn.Linear(512, 512)
            self.head_projection_1 = nn.Linear(512, 512)
            self.head_projection_2 = nn.Linear(512, 512)

        
        

    def forward(self, x):
        
        x_enc = self.encoder(x)

        if self.mode == 'simCLR':
            #x_enc = nn.functional.relu(x_enc)
            #x_enc = nn.functional.adaptive_avg_pool3d(x_enc, 1)
            #x_enc = nn.functional.adaptive_avg_pool2d(x_enc, 1)
            x_enc = torch.flatten(x_enc, 1)
            x_enc = self.hidden_representation(x_enc)
            x_enc = nn.functional.relu(x_enc)
            x_enc = self.head_projection_1(x_enc)
            x_enc = nn.functional.relu(x_enc)
            x_enc = self.head_projection_2(x_enc)
            return x_enc
        
        elif self.mode == "classif":
            x = torch.flatten(x_enc, 1)
            x = self.relu(self.fc1(x))
            x = self.relu(self.fc2(x))
            return x.squeeze(1)

       


class Classifier(nn.Module):
    def __init__(self, nb_classes, features):
        super(Classifier, self).__init__()
        self.num_classes = nb_classes
        self.features = features
        self.fc1 = nn.Linear(512, 512, bias=True)
        self.fc2 = nn.Linear(512, self.num_classes, bias=True)
        self.relu = nn.LeakyReLU(inplace=True)
        #self.avgpool = nn.AdaptiveAvgPool3d(1)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return x.squeeze(1)




In [10]:
net = ResNet(4, mode="simCLR")

## 3. Loss Definition

In [11]:
import logging
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as func
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np

class GeneralizedSupervisedNTXenLoss(nn.Module):
    def __init__(self, kernel='rbf', temperature=0.1, return_logits=False, sigma=1.0):
       
        super().__init__()
        self.kernel = kernel
        self.sigma = sigma
        if self.kernel == 'rbf':
            self.kernel = lambda y1, y2: rbf_kernel(y1, y2, gamma=1./(2*self.sigma**2))
        else:
            assert hasattr(self.kernel, '__call__'), 'kernel must be a callable'
        self.temperature = temperature
        self.return_logits = return_logits
        self.INF = 1e8

    def forward(self, z_i, z_j, labels):
        N = len(z_i)
        z_i = func.normalize(z_i, p=2, dim=-1) 
        z_j = func.normalize(z_j, p=2, dim=-1) 
        sim_zii= (z_i @ z_i.T) / self.temperature 
        sim_zjj = (z_j @ z_j.T) / self.temperature 
        sim_zij = (z_i @ z_j.T) / self.temperature 
        sim_zii = sim_zii - self.INF * torch.eye(N, device=z_i.device)
        sim_zjj = sim_zjj - self.INF * torch.eye(N, device=z_i.device)

        all_labels = labels.view(N, -1).repeat(2, 1).detach().cpu().numpy() # [2N, *]
        weights = self.kernel(all_labels, all_labels) 
        weights = weights * (1 - np.eye(2*N)) 
        weights /= weights.sum(axis=1)
        sim_Z = torch.cat([torch.cat([sim_zii, sim_zij], dim=1), torch.cat([sim_zij.T, sim_zjj], dim=1)], dim=0) # [2N, 2N]
        log_sim_Z = func.log_softmax(sim_Z, dim=1)

        loss = -1./N * (torch.from_numpy(weights).to(z_i.device) * log_sim_Z).sum()

        correct_pairs = torch.arange(N, device=z_i.device).long()

        if self.return_logits:
            return loss, sim_zij, correct_pairs

        return loss

    def __str__(self):
        return "{}(temp={}, kernel={}, sigma={})".format(type(self).__name__, self.temperature,
                                                         self.kernel.__name__, self.sigma)



class NTXenLoss(nn.Module):

    def __init__(self, temperature=0.1, return_logits=False):
        super().__init__()
        self.temperature = temperature
        self.INF = 1e8
        self.return_logits = return_logits

    def forward(self, z_i, z_j):
        N = len(z_i)
        z_i = func.normalize(z_i, p=2, dim=-1) # dim [N, D]
        z_j = func.normalize(z_j, p=2, dim=-1) # dim [N, D]
        sim_zii= (z_i @ z_i.T) / self.temperature # dim [N, N] => Upper triangle contains incorrect pairs
        sim_zjj = (z_j @ z_j.T) / self.temperature # dim [N, N] => Upper triangle contains incorrect pairs
        sim_zij = (z_i @ z_j.T) / self.temperature # dim [N, N] => the diag contains the correct pairs (i,j) (x transforms via T_i and T_j)
        # 'Remove' the diag terms by penalizing it (exp(-inf) = 0)
        sim_zii = sim_zii - self.INF * torch.eye(N, device=z_i.device)
        sim_zjj = sim_zjj - self.INF * torch.eye(N, device=z_i.device)
        correct_pairs = torch.arange(N, device=z_i.device).long()
        loss_i = func.cross_entropy(torch.cat([sim_zij, sim_zii], dim=1), correct_pairs)
        loss_j = func.cross_entropy(torch.cat([sim_zij.T, sim_zjj], dim=1), correct_pairs)

        if self.return_logits:
            return (loss_i + loss_j), sim_zij, correct_pairs

        return (loss_i + loss_j)

    def __str__(self):
        return "{}(temp={})".format(type(self).__name__, self.temperature)

In [12]:
train_loss = NTXenLoss(temperature=config.temperature,return_logits=False)
val_loss =  GeneralizedSupervisedNTXenLoss(temperature=config.temperature,
                                              kernel='rbf',
                                              sigma=config.sigma,
                                              return_logits=True)

In [13]:
def KNN(emb, cls, batch_size):
    Ks=[1, 10, 50, 100]
    preds = []
    mask = torch.eye(batch_size).bool().cuda()
    mask = F.pad(mask, (0, len(emb) - batch_size))
    for batch_x in torch.split(emb, batch_size):
        dist = torch.norm(
            batch_x.unsqueeze(1) - emb.unsqueeze(0), dim=2, p="fro")
        now_batch_size = len(batch_x)
        mask = mask[:now_batch_size]
        dist = torch.masked_fill(dist, mask, float('inf'))
        # update mask
        mask = F.pad(mask[:, :-now_batch_size], (now_batch_size, 0))
        pred = []
        for K in Ks:
            knn = dist.topk(K, dim=1, largest=False).indices
            knn = cls[knn].cpu()
            pred.append(torch.mode(knn).values)
            
            
        pred = torch.stack(pred, dim=0)
        preds.append(pred)
    preds = torch.cat(preds, dim=1)
    accs = [(pred == cls.cpu()).float().mean().item() for pred in preds]
    return max(accs)

## 4. Training

In [14]:
from tqdm import tqdm
from torch.nn import DataParallel
from torch.nn import CrossEntropyLoss

class yAwareCLModel:

    def __init__(self, net, train_loss, val_loss, loader_train, loader_val, config, scheduler=None):

        super().__init__()
        self.logger = logging.getLogger("yAwareCL")
        self.train_loss = train_loss
        self.val_loss = val_loss
        self.fine_loss = CrossEntropyLoss()
        self.model = net
        self.optimizer = torch.optim.Adam(net.parameters(), lr=config.lr, weight_decay=config.weight_decay)
        self.scheduler = scheduler
        self.loader = loader_train
        self.loader_val = loader_val
        self.device = torch.device("cuda" if config.cuda else "cpu")
        if config.cuda and not torch.cuda.is_available():
            raise ValueError("No GPU found: set cuda=False parameter.")
        self.config = config
        self.metrics = {}

        
        self.model = DataParallel(self.model).to(self.device)

    def pretraining(self):
        #print(self.loss)
        #print(self.optimizer)
        max_acc = -1
        min_loss = 100
        for epoch in tqdm(range(self.config.nb_epochs)):

            ## Training step
            self.model.train()
            nb_batch = len(self.loader)
            batch_num = len(self.loader_val)
            training_loss = 0
            #pbar = tqdm(total=nb_batch, desc="Training")
            #for (inputs, labels) in self.loader:
            for inputs in self.loader:
                #pbar.update()
                inputs = inputs.to(self.device)
                #labels = labels.to(self.device)
                self.optimizer.zero_grad()
                z_i = self.model(inputs[:, 0, :])
                z_j = self.model(inputs[:, 1, :])
                #print('train', inputs[:, 0, :].size(), ' ', z_i.size())
                #batch_loss, logits, target = self.loss(z_i, z_j, labels)
                batch_loss = self.train_loss(z_i, z_j)
                batch_loss.backward()
                self.optimizer.step()
                training_loss += float(batch_loss) / nb_batch
            #pbar.close()

            ## Validation step
            nb_batch = len(self.loader_val)
            val_loss = 0
            val_acc = 0
            val_values = {}
            with torch.no_grad():
                self.model.eval()
                for (inputs) in self.loader_val:
                    inputs = inputs.to(self.device)
                    z_i = self.model(inputs[:, 0, :])
                    z_j = self.model(inputs[:, 1, :])
                    batch_loss = self.train_loss(z_i, z_j)
                    val_loss += float(batch_loss) / nb_batch
                    
                """for (inputs, labels) in self.loader_val:
                  inputs = inputs.to(self.device)
                  labels = labels.to(self.device)
                  z_i = self.model(inputs[:, 0, :])
                  acc = KNN(z_i, labels, config.batch_size)
                  val_acc += float(acc) / batch_num  """    
            print("Epoch [{}/{}] Training loss = {:.4f}\t Validation Loss = {:.4f}\t".format(epoch+1, self.config.nb_epochs, training_loss, val_loss))
            #metrics = "\t".join(["Validation {}: {:.4f}".format(m, v) for (m, v) in val_values.items()])
            #print("Epoch [{}/{}] Training loss = {:.4f}\t Validation loss = {:.4f}\t".format(
            #    epoch+1, self.config.nb_epochs, training_loss, val_loss)+metrics, flush=True)

            if self.scheduler is not None:
                self.scheduler.step()

            if (epoch % self.config.nb_epochs_per_saving == 0 or epoch == self.config.nb_epochs - 1) and epoch>5 and val_loss<min_loss:
                min_loss = val_loss
                torch.save({
                    "epoch": epoch,
                    "model": self.model.state_dict(),
                    "optimizer": self.optimizer.state_dict()},
                    os.path.join(self.config.checkpoint_dir, "{val_acc}_epoch_{epoch}.pth".
                                 format(val_acc=val_acc, epoch=epoch)))


    def fine_tuning(self):
        loss = CrossEntropyLoss()
        max_acc = -1
        min_loss = 100
        batch_num = len(self.loader)
        for epoch in tqdm(range(10)):
            ## Training step
            self.model.train()
            nb_batch = len(self.loader)
            training_loss = 0
            for (inputs, labels) in self.loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                self.optimizer.zero_grad()
                y = self.model(inputs)
                #print("y", y.size(), "label", labels.size())
                batch_loss = loss(y,labels)
                batch_loss.backward()
                #print('batch', batch_loss.detach().cpu().numpy())
                self.optimizer.step()
                training_loss +=  np.float32(batch_loss.detach().cpu().numpy()) / nb_batch

            ## Validation step
            
            val_acc = 0
            val_loss = 0
            with torch.no_grad():
                self.model.eval()
                for (inputs, labels) in self.loader_val:
                  inputs = inputs.to(self.device)
                  labels = labels.to(self.device)
                  y = self.model(inputs)
                  batch_loss = loss(y,labels)
                  val_loss +=  np.float32(batch_loss.detach().cpu().numpy()) / nb_batch
                  #print('test', inputs.size(), ' ', z_i.size())
                  acc = KNN(y, labels, config.batch_size)
                  val_acc += float(acc) / len(self.loader_val) 
            
            print("Epoch [{}/{}] Training loss = {:.4f}\t Validation Loss = {:.4f}\t Validation Accuracy = {:.4f}\t".format(epoch+1, self.config.nb_epochs, training_loss, val_loss, val_acc))
            torch.save({
                    "epoch": epoch,
                    "model": self.model.state_dict(),
                    "optimizer": self.optimizer.state_dict()},
                    os.path.join(self.config.checkpoint_dir, "Finetune_{max_acc}_epoch_{epoch}.pth".
                                 format(max_acc=max_acc, epoch=epoch)))
            """if (epoch % self.config.nb_epochs_per_saving == 0 or epoch == self.config.nb_epochs - 1) and val_acc>max_acc:
                max_acc = val_acc
                torch.save({
                    "epoch": epoch,
                    "model": self.model.state_dict(),
                    "optimizer": self.optimizer.state_dict()},
                    os.path.join(self.config.checkpoint_dir, "Finetune_{max_acc}_epoch_{epoch}.pth".
                                 format(max_acc=max_acc, epoch=epoch)))"""

            if self.scheduler is not None:
                self.scheduler.step()
    def test1(self):
        val_acc = 0
        batch_num = len(self.loader_val)
        with torch.no_grad():
            self.model.eval()
            for (inputs, labels) in self.loader_val:
              inputs = inputs.to(self.device)
              labels = labels.to(self.device)
              z_i = self.model(inputs)
              #print('test', inputs.size(), ' ', z_i.size())
              acc = KNN(z_i, labels, config.batch_size)
              val_acc += float(acc) / batch_num 
        
        print("KNN Accuracy = {:.4f}\t".format(val_acc))
    def test2(self):
        loss = 0
        batch_num = len(self.loader)
        with torch.no_grad():
            self.model.eval()
            for (inputs) in self.loader:
              inputs = inputs.to(self.device)
              z_i = self.model(inputs[:, 0, :])
              z_j = self.model(inputs[:, 1, :])
              #print('train', inputs[:, 0, :].size(), ' ', z_i.size())
              #batch_loss, logits, target = self.loss(z_i, z_j, labels)
              batch_loss = self.train_loss(z_i, z_j)
              loss += float(batch_loss) / len(self.loader)
        
        print("Loss = {:.4f}\t".format(loss))   
    def load_model(self, path):
            checkpoint = None
            checkpoint = torch.load(path, map_location=lambda storage, loc: storage)
            unexpected = self.model.load_state_dict(checkpoint["model"], strict=False)
            self.logger.info('Model loading info: {}'.format(unexpected))
    

In [16]:
model = yAwareCLModel(net, train_loss, val_loss, train_loader, val_loader, config)
model.pretraining()

  0%|          | 1/200 [01:14<4:05:46, 74.10s/it]

Epoch [1/200] Training loss = 1.2666	 Validation Loss = 0.7806	


  1%|          | 2/200 [02:28<4:05:20, 74.34s/it]

Epoch [2/200] Training loss = 0.9456	 Validation Loss = 0.7000	


  2%|▏         | 3/200 [03:43<4:04:26, 74.45s/it]

...

Epoch [198/200] Training loss = 0.1434	 Validation Loss = 0.2493	


100%|█████████▉| 199/200 [4:07:22<01:14, 74.57s/it]

Epoch [199/200] Training loss = 0.1280	 Validation Loss = 0.2335	


100%|██████████| 200/200 [4:08:37<00:00, 74.59s/it]

Epoch [200/200] Training loss = 0.1492	 Validation Loss = 0.2554	


## 5. Finetuning

In [21]:
from sklearn.model_selection import train_test_split
class FineTuneDataSet(Dataset):
    def __init__(self, main_dir, transform, mode):
        self.main_dir = main_dir
        self.transform = transform
        if mode=="train":
          all_imgs = os.listdir(main_dir)
        elif mode=="test":
          all_imgs=[]
          for path, subdirs, files in os.walk(main_dir):
              for name in files:
                  all_imgs.append(os.path.join(path, name))
        self.total_imgs = natsort.natsorted(all_imgs)
        self.mode = mode
    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        if self.mode=="train":
          img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        elif self.mode=="test":
          img_loc=self.total_imgs[idx]
        image = Image.open(img_loc)#.convert("RGB")
        
        if self.mode=="test":
          labels = os.path.basename(os.path.dirname(img_loc)) 
          x = self.transform(image)
          x = torch.from_numpy(np.asarray(x))
          #print('x.shape', x.shape)
          #x = x.reshape(1, x.shape[0], x.shape[1], x.shape[2])
          labels = torch.from_numpy(np.asarray(int(labels)))
          return (x, labels)
        else:
          x1 = self.transform(image)
          x2 = self.transform(image)
          x = np.stack((x1, x2), axis=0)
          x = torch.from_numpy(np.asarray(x))
          return x

test_set = FineTuneDataSet("hw2/test", transform=train_transform, mode = "test")
training_data, testing_data = train_test_split(test_set, test_size=0.2, random_state=25)
train_loader = DataLoader(
    training_data,
    batch_size=config.batch_size,
    sampler=RandomSampler(training_data),
    #collate_fn=train_set.collate_fn,
    pin_memory=config.pin_mem,
    num_workers=config.num_cpu_workers
                              
)
val_loader = DataLoader(
    testing_data,
    batch_size=config.batch_size,
    sampler=RandomSampler(testing_data),
    #collate_fn=test_set.collate_fn,
    pin_memory=config.pin_mem,
    num_workers=config.num_cpu_workers
)
net = ResNet(4, mode="simCLR")

model = yAwareCLModel(net, train_loss, val_loss, train_loader, val_loader, config)
model.load_model('save1/0_epoch_163.pth')
model.model.head_projection_1 = nn.Linear(512, 4) 
model.model.head_projection_2 = nn.Identity()
model.fine_tuning()

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/200] Training loss = 6.1628	 Validation Loss = 1.5102	 Validation Accuracy = 0.5500	


 10%|█         | 1/10 [00:03<00:30,  3.38s/it]

Epoch [2/200] Training loss = 5.9503	 Validation Loss = 1.4432	 Validation Accuracy = 0.6500	


 20%|██        | 2/10 [00:06<00:27,  3.43s/it]

...

 80%|████████  | 8/10 [00:27<00:06,  3.41s/it]

Epoch [9/200] Training loss = 0.5402	 Validation Loss = 0.1332	 Validation Accuracy = 0.9400	


 90%|█████████ | 9/10 [00:30<00:03,  3.40s/it]

Epoch [10/200] Training loss = 0.5869	 Validation Loss = 0.0809	 Validation Accuracy = 0.9200	


100%|██████████| 10/10 [00:34<00:00,  3.41s/it]


# **Evaluation & Save Embedding**

In [25]:
import natsort
from torch.utils.data import DataLoader, Dataset, RandomSampler

class EvalDataSet(Dataset):
    def __init__(self, main_dir, transform, mode):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)
    def __len__(self):
        return len(self.total_imgs)

   
        
    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc)#.convert("RGB")
        x = self.transform(image)
        x = torch.from_numpy(np.asarray(x))
        x = x.reshape(1, x.shape[0], x.shape[1], x.shape[2])
        return x
        
test_transform = transforms.Compose([ 
    #transforms.Grayscale(1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
eval_set = EvalDataSet("hw2/unlabeled", transform=test_transform, mode = "test")


eval_loader = DataLoader(
    eval_set,
    batch_size=128,
    #collate_fn=test_set.collate_fn,
    pin_memory=config.pin_mem,
    num_workers=config.num_cpu_workers, 
    shuffle=False
)

In [30]:
net = ResNet(4, mode="simCLR")
net = net.cuda()
checkpoint = torch.load("save1/0_epoch_163.pth", map_location=lambda storage, loc: storage)
unexpected = net.load_state_dict(checkpoint["model"], strict=False)

embedding = np.empty((0, 512),dtype="float32")
for inputs in eval_loader:
  inputs = inputs.cuda()
  y = net(inputs[:, 0, :])
  #print(y.size())
  embedding = np.concatenate((embedding, y.detach().cpu().numpy()))
  #print(embedding.shape)
result = np.zeros((7294, 512),dtype="float32")
result[:embedding.shape[0],:embedding.shape[1]] = embedding
np.save('0710749_noft_0_15.npy', result)
print('complete')

complete


In [20]:
test_set = FineTuneDataSet("hw2/test", transform=test_transform, mode = "test")

val_loader = DataLoader(
    test_set,
    batch_size=config.batch_size,
    sampler=RandomSampler(test_set),
    #collate_fn=test_set.collate_fn,
    pin_memory=config.pin_mem,
    num_workers=config.num_cpu_workers
)

net = ResNet(4, mode="simCLR")
model = yAwareCLModel(net, train_loss, val_loss, train_loader, val_loader, config)
model.load_model('save1/0_epoch_163.pth')
model.test1()

KNN Accuracy = 0.5331	


In [34]:
train_transform = transforms.Compose([
    #transforms.RandomOrder(train_transform_set),
    #transforms.Grayscale(1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_set = CustomDataSet("hw2/unlabeled", transform=train_transform, mode = "train")

train_loader = DataLoader(
    train_set,
    batch_size=config.batch_size,
    sampler=RandomSampler(train_set),
    #collate_fn=train_set.collate_fn,
    pin_memory=config.pin_mem,
    num_workers=config.num_cpu_workers,
    shuffle = False                          
)
net = ResNet(4, mode="simCLR")
model = yAwareCLModel(net, train_loss, val_loss, train_loader, val_loader, config)
model.load_model('save1/0_epoch_163.pth')
model.test2()

Loss = 0.1156	


In [32]:
a = np.load("0710749_noft_0_15.npy")

In [33]:
a.shape

(7294, 512)